# Pipeline for generating a question to further prompt the user.


In [2]:
import pandas as pd
import os
import json
from tqdm import trange
import time
import re

from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from groq import Groq


## load env variables
GROQ_API_KEY       = os.environ["GROQ_API_KEY"]
CHAT_MODEL         = "llama3-70b-8192"
client = Groq()

In [8]:
def load_data(index):
    with open("../data/further_prompting/combined_data.json", "r", encoding="utf-8") as fin:
        data = json.load(fin)
        
    return data[index]

{'QA_pair_number': 0,
 'Question': 'What is the percentage of total mortality in Singapore attributed to ischemic heart disease, based on national health statistics from the Ministry of Health in 2019?',
 'Answer': 'Ischemic heart disease contributes to 18.8% of total mortality in Singapore, based on national health statistics from the Ministry of Health in 2019.',
 '3/10': ["I think it's around 20% or something, I'm not really sure.",
  "Ischemic heart disease is a big deal in Singapore, but I don't have the exact number.",
  "Heart disease is a major killer in Singapore, but I'm not sure what the exact percentage is.",
  "I'm pretty sure it's higher than 10%, but I don't have the exact stat.",
  "Singapore has a lot of heart problems, but I don't know the exact mortality rate."],
 '6/10': ["According to the Ministry of Health's 2019 statistics, ischemic heart disease accounts for nearly 19% of total mortality in Singapore.",
  'In 2019, ischemic heart disease was responsible for appr

In [15]:
def extract_answer(answer, pattern=r'"guiding_question"\s*:\s*"([^"]+)"'):
    """
    Extracts the answer from the text associated with the key "guiding_question".
    
    Args:
        text (str): The input string to search for the guiding question.
    
    Returns:
        str: The extracted guiding question or an error message.
    """
    try:        
        # Search for the pattern in the input text
        match = re.search(pattern, answer)
        
        # If a match is found, return the captured group (the question)
        if match:
            return match.group(1)
        else:
            # If no match is found, return an error message
            return "No guiding question found in the input text."
    
    except Exception as e:
        # If any unexpected error occurs, return a generic error message
        return f"An error occurred: {str(e)}"
        

PROMPT = \
'''You are an expert in linguistic variation and medical communication. 
You are provided with a medical question, a persons answer to that question, and the model answer to that question. 
Your task is to generate a follow up question that aids the user in the answering of the question. 
You are to focus in on what is missing from the users answer in comparison to the model answer, and hint that to the user in the follow up question.
You are to reply in a cheerful, friendly tone.

Question:
{question}

Users Answer:
{user_ans}

Model answer:
{model_ans}

The returning format should be like this:
{{"guiding_question": "Question to help the user better answer the question"}}

Before returning the answer, ensure and double check that the answer is in accordance to the format above.
'''

def generate_guiding_question(qn, user_ans, model_ans) -> str:
    '''Generates the question used to re-prompt the user, 
    helping him to answer the question better with some hints'''
    
    prompt = PromptTemplate(
            template=PROMPT,
            input_variables=["question", "user_ans", "model_ans"]
        ) 
    
    final_prompt = prompt.format(question=qn, user_ans=user_ans, model_ans=model_ans)

    completion = client.chat.completions.create(
            model=CHAT_MODEL,
            messages=[
                {
                    "role": "user",
                    "content": final_prompt
                }
            ],
            temperature=0,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None
        )

    llm_answer = ''''''
    for chunk in completion:
        llm_answer += chunk.choices[0].delta.content or ""
    # Improve error handling here
    guiding_qn = extract_answer(llm_answer)
    print("===================================")
    print(f"Question:\n{qn}\n")
    print("===================================")
    print(f"User's Answer:\n{user_ans}\n")
    print("===================================")
    print(f"Guiding Question:\n{guiding_qn}\n")
    print("===================================")
    return guiding_qn

data = load_data(1)
model_ans = data['Answer']
qn = data['Question']
ans_index = 1
user_ans = data['3/10'][ans_index]
ans = generate_guiding_question(qn, user_ans, model_ans)


Question:
In what setting do the majority of cardiac arrests occur, according to a study conducted in Singapore in 2019?

User's Answer:
Cardiac arrests usually happen in emergency rooms, right?

Guiding Question:
Hmm, that's a good guess, but what about outside of the hospital setting? Are there any other places where cardiac arrests might be more likely to happen?



In [ ]:
# TODO for Ethan (Generate answer similarity pipeline)
def get_answer_similarity(retry_ans, model_ans) -> float: 
    '''
    Takes in the users retried answer and the model answer. 
    Uses finetuned cross encoder to generate similarity score
    '''
    score = None
    return score

## Discard


In [ ]:

def re_prompt_user(guiding_qn) -> str:
    '''Function to send the guiding question to the frontend from the backend AND retrieve answer from user'''
    # Send the guiding question to the user
    pass
    
def retrieve_retry_answer() -> str:
    # receives the answer back from the user, prob a fetch from the backend or from frontend?
    retry_ans = None
    
    return retry_ans



def retry_pipeline(qn, user_ans, model_ans, threshold=0.7) -> dict:
    status = {"attempt": "FAIL", "retries": 0}
    
    # number of retries set to 2
    for i in range(1,3):
        
        # generate the guiding question with help of LLM
        guiding_qn = generate_guiding_question(qn, user_ans, model_ans)
        
        # Send the guiding question to backend/ somewhere
        re_prompt_user(guiding_qn)
        
        # retrieve the users retried answer
        retry_ans = retrieve_retry_answer()
            
        # Compare with threshold
        score = get_answer_similarity(retry_ans, model_ans)
            
        # if user passes this round
        if score >= threshold:
            status['attempt'] = "PASS"
            return status  
             
        status['retries'] = i
        
    return status